In [1]:
# !pip uninstall --yes numpy
# !pip install numpy<2.0

import numpy
numpy.__version__

'1.26.4'

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

# GA optimization

In [3]:
import pandas as pd
#read inputs 
print('Reading and processing RS inputs..')
data = pd.read_csv('FTstatesDataset_Toolik_0Days.csv', header=0)
data

Reading and processing RS inputs..


,Date,Tair,T0,T8,T16,T23,T31,T38,T46,T61,...,LWLAND,TS,QV2M,SLP,Y,M,D,W,H,S
0,01-Jan-2000,-23.570833,-20.590833,-16.600833,-13.566667,-12.496667,-11.880417,-10.997917,-10.276250,-9.123333,...,-24.548771,-28.508673,0.000304,102747.171875,2000,1,1,1,0,3
1,02-Jan-2000,-26.929583,-20.388750,-16.039583,-13.046667,-12.066250,-11.540000,-10.781667,-10.152500,-9.127500,...,-19.873007,-29.612051,0.000306,102558.156250,2000,1,2,1,0,3
2,03-Jan-2000,-30.194167,-23.720833,-17.707500,-13.665000,-12.415417,-11.740000,-10.872083,-10.207083,-9.167917,...,-21.266411,-42.819632,0.000261,102112.085938,2000,1,3,0,0,3
3,04-Jan-2000,-32.678333,-25.611250,-18.949167,-14.417500,-13.018333,-12.226667,-11.247083,-10.501250,-9.361667,...,-13.013476,-39.689627,0.000229,101606.335938,2000,1,4,0,0,3
4,05-Jan-2000,-33.580000,-25.935417,-19.502917,-14.986667,-13.555000,-12.720417,-11.680417,-10.876250,-9.651667,...,-14.391212,-37.193289,0.000201,101824.546875,2000,1,5,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6075,21-Aug-2016,9.063667,8.438750,8.369542,8.355458,8.627125,8.341542,7.483625,4.450167,1.683333,...,-37.395847,7.405542,0.006165,100375.648438,2016,8,21,1,0,1
6076,22-Aug-2016,2.546250,2.724000,2.705667,2.759250,3.098000,3.168833,3.545458,3.151833,1.530292,...,-26.156282,5.915094,0.005882,100044.414062,2016,8,22,0,0,1
6077,23-Aug-2016,5.623292,6.711625,6.689875,6.744417,7.474208,7.509583,6.049917,2.905083,1.041417,...,-21.981482,2.444971,0.004555,101036.132812,2016,8,23,0,0,1
6078,24-Aug-2016,13.034167,12.631667,12.531250,12.532917,12.897917,12.571250,10.608458,4.953542,1.244083,...,-52.502403,9.777917,0.007226,101742.593750,2016,8,24,0,0,1


In [33]:
import math
# trainset = pd.read_csv(data_path, header=0, usecols=[target] + ftrz)
# if ~all(isinstance(x, int) for x in trainset[target]):
#     trainset[f"{target}"] = pd.cut(trainset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

# trainset.loc[:, trainset.columns != target].values

In [8]:
def modellearning(trgt, selected_features, trainset, testset):
    # Check if the selected features list is empty
    if len(selected_features) == 0:
        print("No features selected. Skipping model training.")
        return -float('inf')  # Return a low score to signal this case in the GA

    # Read train and test sets with specific columns 
    trainset = trainset[selected_features + [trgt]]
    testset = testset[selected_features + [trgt]]

    # Extract X and y
    X_train, X_test = trainset.loc[:, trainset.columns != trgt].values, testset.loc[:, testset.columns != trgt].values
    y_train, y_test = np.squeeze(trainset[trgt].values), np.squeeze(testset[trgt].values)

    # Scale features
    print(X_train.shape, X_test.shape)
    X_train_scaled, X_test_scaled = scale_features(X_train, X_test)  

    # Train final ExtraTreesClassifier model under finetuned parameters
    model_final = ExtraTreesClassifier(n_jobs=-1, random_state=42)
    
    model_final.fit(X_train_scaled, y_train)

    # Make predictions
    y_pred_test = model_final.predict(X_test_scaled)  # Reshape the data to 2D if needed
    
    # Evaluate the model
    accuracy_test = balanced_accuracy_score(y_test, y_pred_test)
    return accuracy_test

# Function to scale features
def scale_features(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

In [9]:
# !pip install deap
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms
import random
import math
import re

# Dictionary to store ACCURACY for evaluated feature combinations
evaluated_combinations = {}

# Objective function for GA
def evalOneMax(individual, all_feature_names, target):
    selected_features = [all_feature_names[i] for i, is_selected in enumerate(individual) if is_selected]
#     print(selected_features, end=':')
    selected_features_tuple = tuple(sorted(selected_features))
    
    # Ensure at least 1 feature are selected
    if len(selected_features) < 0:
        return -float('inf'),
    
    # Always proceed with model training, bypassing the cache check
    acc = modellearning(target, selected_features, trainset, testset)
#     print(np.round(acc*100,3), end=',')#
    return (acc,)

def optimize_features(target, all_feature_names, nmbr_population, nmbr_generation, known_good_features=None):
    best_features = []  # Initialize to an empty list
    best_acc = -float('inf')  # Initialize to negative infinity
    
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(all_feature_names))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evalOneMax, all_feature_names=all_feature_names, target=target)

    population = toolbox.population(n=nmbr_population)

    # Seed the population with a known good individual, if provided
    if known_good_features:
        good_individual = [0] * len(all_feature_names)
        for feature in known_good_features:
            idx = all_feature_names.index(feature)
            good_individual[idx] = 1
        population[0] = creator.Individual(good_individual)
        population[0].fitness.values = evalOneMax(population[0], all_feature_names, target)

    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=nmbr_generation, verbose=1)

    # Extract the best individual
    best_ind = tools.selBest(population, k=1)[0]
    best_features = [all_feature_names[i] for i, is_selected in enumerate(best_ind) if is_selected]
    best_acc = best_ind.fitness.values[0]
    
    return best_features, best_acc


# Toolik lake

In [6]:
# load data
target = 'T0'
data_path = f"FTstates_Toolik_{target}_Trainset_0Days.csv"

ftrz = ['Y','M','D','W','H','S','SNODP','SWGDN','LWGAB','T2M','SWLAND','GHTSKIN','HFLUX','SPEED','TLML','TSH','EVPSOIL','LWLAND','TS','QV2M','SLP']

#trainset
trainset = pd.read_csv(data_path, header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in trainset[target]):
    trainset[f"{target}"] = pd.cut(trainset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#testset
testset = pd.read_csv(data_path.replace("Train", "Test"), header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in testset[target]):
    testset[f"{target}"] = pd.cut(testset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#other configurations
prdHrz = int(re.search(r'(\d+)Days', data_path).group(1))
all_feature_names = ftrz
known_good_features = ['GHTSKIN']  # Replace maybe

# Initial GA Run to find a smaller subset of features
# nmbr_population_initial = 1500
# nmbr_generation_initial = 100
# print(f'*** Initial optimization: +{prdHrz}Days')          
# initial_best_features, initial_best_acc = optimize_features(target, all_feature_names, nmbr_population_initial, nmbr_generation_initial, known_good_features)
# print(f"\nInitial Best Features: {initial_best_features}; Initial Best Accuracy: {initial_best_acc}")
initial_best_features, initial_best_acc = ['Y', 'M', 'D', 'W', 'H', 'S', 'T2M', 'GHTSKIN', 'HFLUX'], 0.989063568010936

# Refinement GA Run on the smaller subset of features
nmbr_population_refinement = 10
nmbr_generation_refinement = 50
print(f'*** Refinement optimization: +{prdHrz}Days')
refined_best_features, refined_best_acc = optimize_features(target, initial_best_features, nmbr_population_refinement, nmbr_generation_refinement)
print(f"\nRefined Best Features: {refined_best_features}; Refined Best Accuracy: {refined_best_acc}")

*** Refinement optimization: +0Days
(4864, 4) (1216, 4)
(4864, 4) (1216, 4)
(4864, 3) (1216, 3)
(4864, 3) (1216, 3)
(4864, 2) (1216, 2)
(4864, 5) (1216, 5)
(4864, 6) (1216, 6)
(4864, 2) (1216, 2)
(4864, 6) (1216, 6)
(4864, 5) (1216, 5)
gen	nevals
0  	10    
(4864, 6) (1216, 6)
(4864, 6) (1216, 6)
(4864, 5) (1216, 5)
(4864, 6) (1216, 6)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
1  	7     
(4864, 5) (1216, 5)
(4864, 6) (1216, 6)
2  	2     
(4864, 6) (1216, 6)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
3  	4     
(4864, 5) (1216, 5)
(4864, 4) (1216, 4)
(4864, 6) (1216, 6)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
4  	8     
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
5  	4     
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
(4864, 5) (1216, 5)
6  	6     
(4864, 4) (1216, 4)
(4864, 5) (1216, 5)
(4864, 5) (1216,

In [ ]:
# load data
target = 'T8'
data_path = f"FTstates_Toolik_{target}_Trainset_0Days.csv"

ftrz = ['Y','M','D','W','H','S','SNODP','SWGDN','LWGAB','T2M','SWLAND','GHTSKIN','HFLUX','SPEED','TLML','TSH','EVPSOIL','LWLAND','TS','QV2M','SLP']

#trainset
trainset = pd.read_csv(data_path, header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in trainset[target]):
    trainset[f"{target}"] = pd.cut(trainset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#testset
testset = pd.read_csv(data_path.replace("Train", "Test"), header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in testset[target]):
    testset[f"{target}"] = pd.cut(testset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#other configurations
prdHrz = int(re.search(r'(\d+)Days', data_path).group(1))
all_feature_names = ftrz
known_good_features = ['GHTSKIN']  # Replace maybe

# # Initial GA Run to find a smaller subset of features
nmbr_population_initial = 1500
nmbr_generation_initial = 100
print(f'*** Initial optimization: +{prdHrz}Days')          
initial_best_features, initial_best_acc = optimize_features(target, all_feature_names, nmbr_population_initial, nmbr_generation_initial, known_good_features)
print(f"\nInitial Best Features: {initial_best_features}; Initial Best Accuracy: {initial_best_acc}")
# initial_best_features, initial_best_acc = ['Y', 'S', 'SNODP', 'SWLAND', 'GHTSKIN', 'SPEED', 'TLML', 'EVPSOIL', 'SLP'], 0.9883275092215104

# # Refinement GA Run on the smaller subset of features
# nmbr_population_refinement = 200
# nmbr_generation_refinement = 100
# print(f'*** Refinement optimization: +{prdHrz}Days')
# refined_best_features, refined_best_acc = optimize_features(target, initial_best_features, nmbr_population_refinement, nmbr_generation_refinement)
# print(f"\nRefined Best Features: {refined_best_features}; Refined Best Accuracy: {refined_best_acc}")

*** Initial optimization: +0Days


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	1499  
1  	932   
2  	962   
3  	925   
4  	975   
5  	884   
6  	879   
7  	927   
8  	893   
9  	902   
10 	942   
11 	891   
12 	906   
13 	847   
14 	951   
15 	948   
16 	881   
17 	874   
18 	873   
19 	885   
20 	878   
21 	922   
22 	942   
23 	898   
24 	929   
25 	938   


In [ ]:
# load data
target = 'T16'
data_path = f"FTstates_Toolik_{target}_Trainset_0Days.csv"

ftrz = ['Y','M','D','W','H','S','SNODP','SWGDN','LWGAB','T2M','SWLAND','GHTSKIN','HFLUX','SPEED','TLML','TSH','EVPSOIL','LWLAND','TS','QV2M','SLP']

#trainset
trainset = pd.read_csv(data_path, header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in trainset[target]):
    trainset[f"{target}"] = pd.cut(trainset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#testset
testset = pd.read_csv(data_path.replace("Train", "Test"), header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in testset[target]):
    testset[f"{target}"] = pd.cut(testset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#other configurations
prdHrz = int(re.search(r'(\d+)Days', data_path).group(1))
all_feature_names = ftrz
known_good_features = ['GHTSKIN']  # Replace maybe

# Initial GA Run to find a smaller subset of features
nmbr_population_initial = 1500
nmbr_generation_initial = 100
print(f'*** Initial optimization: +{prdHrz}Days')          
initial_best_features, initial_best_acc = optimize_features(target, all_feature_names, nmbr_population_initial, nmbr_generation_initial, known_good_features)
print(f"\nInitial Best Features: {initial_best_features}; Initial Best Accuracy: {initial_best_acc}")
# initial_best_features, initial_best_acc = ['LC_Type1', 'LC_Type2', 'LC_Type4', 'LC_Type5', 'Latitude', 'Elevation', 'Slope', 'prcp', 'Y', 'W', 'S', 'surface_pressure', 'snowfall_sum', 'snowmelt_sum', 'snow_depth', 'snow_cover'], 0.00295334026115865

# # Refinement GA Run on the smaller subset of features
# nmbr_population_refinement = 200
# nmbr_generation_refinement = 100
# print(f'*** Refinement optimization: +{prdHrz}Days')
# refined_best_features, refined_best_acc = optimize_features(target, initial_best_features, nmbr_population_refinement, nmbr_generation_refinement)
# print(f"\nRefined Best Features: {refined_best_features}; Refined Best Accuracy: {refined_best_acc}")

# Deadhorse

In [4]:
# load data
target = 'T0'
data_path = f"FTstates_Deadhorse_{target}_Trainset_0Days.csv"

ftrz = ['Y','M','D','W','H','S','SNODP','SWGDN','LWGAB','T2M','SWLAND','GHTSKIN','HFLUX','SPEED','TLML','TSH','EVPSOIL','LWLAND','TS','QV2M','SLP']

#trainset
trainset = pd.read_csv(data_path, header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in trainset[target]):
    trainset[f"{target}"] = pd.cut(trainset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#testset
testset = pd.read_csv(data_path.replace("Train", "Test"), header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in testset[target]):
    testset[f"{target}"] = pd.cut(testset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#other configurations
prdHrz = int(re.search(r'(\d+)Days', data_path).group(1))
all_feature_names = ftrz
known_good_features = ['GHTSKIN']  # Replace maybe

# Initial GA Run to find a smaller subset of features
nmbr_population_initial = 1000
nmbr_generation_initial = 50
print(f'*** Initial optimization: +{prdHrz}Days')          
initial_best_features, initial_best_acc = optimize_features(target, all_feature_names, nmbr_population_initial, nmbr_generation_initial, known_good_features)
print(f"\nInitial Best Features: {initial_best_features}; Initial Best Accuracy: {initial_best_acc}")
# initial_best_features, initial_best_acc = ['LC_Type1', 'LC_Type2', 'LC_Type4', 'LC_Type5', 'Latitude', 'Elevation', 'Slope', 'prcp', 'Y', 'W', 'S', 'surface_pressure', 'snowfall_sum', 'snowmelt_sum', 'snow_depth', 'snow_cover'], 0.00295334026115865

# Refinement GA Run on the smaller subset of features
nmbr_population_refinement = 200
nmbr_generation_refinement = 100
print(f'*** Refinement optimization: +{prdHrz}Days')
refined_best_features, refined_best_acc = optimize_features(target, initial_best_features, nmbr_population_refinement, nmbr_generation_refinement)
print(f"\nRefined Best Features: {refined_best_features}; Refined Best Accuracy: {refined_best_acc}")

*** Initial optimization: +0Days


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	999   
1  	569   
2  	614   
3  	618   
4  	600   
5  	578   
6  	582   
7  	597   
8  	605   
9  	598   
10 	598   
11 	579   
12 	598   
13 	602   
14 	620   
15 	630   
16 	615   
17 	610   
18 	625   
19 	606   
20 	583   
21 	614   
22 	608   
23 	590   
24 	604   
25 	589   
26 	614   
27 	597   
28 	609   
29 	607   
30 	621   
31 	592   
32 	613   
33 	603   
34 	573   
35 	569   
36 	625   
37 	590   
38 	577   
39 	568   
40 	590   
41 	617   
42 	612   
43 	619   
44 	583   
45 	601   
46 	628   
47 	598   
48 	614   
49 	599   
50 	605   

Initial Best Features: ['Y', 'M', 'D', 'W', 'H', 'S', 'SNODP', 'SWGDN', 'LWGAB', 'SWLAND', 'GHTSKIN', 'TLML', 'TSH', 'SLP']; Initial Best Accuracy: 0.987189064379681
*** Refinement optimization: +0Days


/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/mohamed.ahajjam/.local/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	200   
1  	97    
2  	107   
3  	131   
4  	125   
5  	134   
6  	113   
7  	112   
8  	128   
9  	111   
10 	129   
11 	116   
12 	124   
13 	111   
14 	121   
15 	108   
16 	107   
17 	104   
18 	123   
19 	110   
20 	136   
21 	102   
22 	111   
23 	117   
24 	122   
25 	119   
26 	130   
27 	142   
28 	116   
29 	108   
30 	122   
31 	134   
32 	113   
33 	107   
34 	123   
35 	96    
36 	115   
37 	120   
38 	114   
39 	111   
40 	116   
41 	120   
42 	121   
43 	130   
44 	115   
45 	122   
46 	127   
47 	108   
48 	117   
49 	122   
50 	126   
51 	123   
52 	116   
53 	113   
54 	112   
55 	132   
56 	119   
57 	124   
58 	109   
59 	128   
60 	115   
61 	110   
62 	122   
63 	126   
64 	119   
65 	122   
66 	133   
67 	119   
68 	121   
69 	123   
70 	138   
71 	121   
72 	110   
73 	111   
74 	118   
75 	113   
76 	99    
77 	111   
78 	129   
79 	112   
80 	108   
81 	119   
82 	104   
83 	119   
84 	120   
85 	125   
86 	110   
87 	126   
88 	110   
89 	117   

In [10]:
# load data
target = 'T_07'
data_path = f"FTstates_Deadhorse_{target}_Trainset_0Days.csv"

ftrz = ['Y','M','D','W','H','S','SNODP','SWGDN','LWGAB','T2M','SWLAND','GHTSKIN','HFLUX','SPEED','TLML','TSH','EVPSOIL','LWLAND','TS','QV2M','SLP']

#trainset
trainset = pd.read_csv(data_path, header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in trainset[target]):
    trainset[f"{target}"] = pd.cut(trainset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#testset
testset = pd.read_csv(data_path.replace("Train", "Test"), header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in testset[target]):
    testset[f"{target}"] = pd.cut(testset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#other configurations
prdHrz = int(re.search(r'(\d+)Days', data_path).group(1))
all_feature_names = ftrz
known_good_features = ['GHTSKIN']  # Replace maybe

# # Initial GA Run to find a smaller subset of features
# nmbr_population_initial = 1000
# nmbr_generation_initial = 50
# print(f'*** Initial optimization: +{prdHrz}Days')          
# initial_best_features, initial_best_acc = optimize_features(target, all_feature_names, nmbr_population_initial, nmbr_generation_initial, known_good_features)
# print(f"\nInitial Best Features: {initial_best_features}; Initial Best Accuracy: {initial_best_acc}")
initial_best_features, initial_best_acc = ['Y', 'M', 'D', 'SNODP', 'EVPSOIL'], 0.9949471340178146

# Refinement GA Run on the smaller subset of features
nmbr_population_refinement = 200
nmbr_generation_refinement = 100
print(f'*** Refinement optimization: +{prdHrz}Days')
refined_best_features, refined_best_acc = optimize_features(target, initial_best_features, nmbr_population_refinement, nmbr_generation_refinement)
print(f"\nRefined Best Features: {refined_best_features}; Refined Best Accuracy: {refined_best_acc}")

*** Refinement optimization: +0Days
(3989, 4) (998, 4)
(3989, 1) (998, 1)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 1) (998, 1)
(3989, 2) (998, 2)
(3989, 1) (998, 1)
(3989, 1) (998, 1)
(3989, 2) (998, 2)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 2) (998, 2)
(3989, 2) (998, 2)
(3989, 3) (998, 3)
(3989, 2) (998, 2)
(3989, 3) (998, 3)
(3989, 1) (998, 1)
(3989, 1) (998, 1)
(3989, 2) (998, 2)
(3989, 5) (998, 5)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 2) (998, 2)
(3989, 3) (998, 3)
(3989, 2) (998, 2)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 2) (998, 2)
(3989, 4) (998, 4)
(3989, 2) (998, 2)
(3989, 1) (998, 1)
(3989, 4) (998, 4)
(3989, 2) (998, 2)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
No features selected. Skipping model training.
(3989, 4) (998, 4)
(3989, 2) (998, 2)
(3989, 1) (998, 1)
(3989, 2) (998, 2)
(3989, 3) (998, 3)
(3989, 2) (998, 2)
(3989

(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 2) (998, 2)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 5) (998, 5)
(3989, 3) (998, 3)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 3) (998, 3)
2  	125   
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(

(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (9

(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 3) (998, 3)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
9  	113   
(3989, 3) (998, 3)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(

(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 3) (998, 3)
(3989, 5) (9

...

(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (9

(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
88 	128   
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(

(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (9

(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 3) (998, 3)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
95 	129   
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(

(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 4) (998, 4)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (998, 5)
(3989, 5) (9

In [11]:
# load data
target = 'T_12'
data_path = f"FTstates_Deadhorse_{target}_Trainset_0Days.csv"

ftrz = ['Y','M','D','W','H','S','SNODP','SWGDN','LWGAB','T2M','SWLAND','GHTSKIN','HFLUX','SPEED','TLML','TSH','EVPSOIL','LWLAND','TS','QV2M','SLP']

#trainset
trainset = pd.read_csv(data_path, header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in trainset[target]):
    trainset[f"{target}"] = pd.cut(trainset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#testset
testset = pd.read_csv(data_path.replace("Train", "Test"), header=0, usecols=[target] + ftrz)
if ~all(isinstance(x, int) for x in testset[target]):
    testset[f"{target}"] = pd.cut(testset[f'{target}'],bins=[-math.inf,0,+math.inf],labels=[0,1])

#other configurations
prdHrz = int(re.search(r'(\d+)Days', data_path).group(1))
all_feature_names = ftrz
known_good_features = ['GHTSKIN']  # Replace maybe

# Initial GA Run to find a smaller subset of features
nmbr_population_initial = 1000
nmbr_generation_initial = 50
print(f'*** Initial optimization: +{prdHrz}Days')          
initial_best_features, initial_best_acc = optimize_features(target, all_feature_names, nmbr_population_initial, nmbr_generation_initial, known_good_features)
print(f"\nInitial Best Features: {initial_best_features}; Initial Best Accuracy: {initial_best_acc}")
# initial_best_features, initial_best_acc = ['LC_Type1', 'LC_Type2', 'LC_Type4', 'LC_Type5', 'Latitude', 'Elevation', 'Slope', 'prcp', 'Y', 'W', 'S', 'surface_pressure', 'snowfall_sum', 'snowmelt_sum', 'snow_depth', 'snow_cover'], 0.00295334026115865

# Refinement GA Run on the smaller subset of features
nmbr_population_refinement = 200
nmbr_generation_refinement = 100
print(f'*** Refinement optimization: +{prdHrz}Days')
refined_best_features, refined_best_acc = optimize_features(target, initial_best_features, nmbr_population_refinement, nmbr_generation_refinement)
print(f"\nRefined Best Features: {refined_best_features}; Refined Best Accuracy: {refined_best_acc}")

*** Initial optimization: +0Days
(3989, 1) (998, 1)
(3989, 10) (998, 10)
(3989, 14) (998, 14)
(3989, 12) (998, 12)
(3989, 8) (998, 8)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 6) (998, 6)
(3989, 6) (998, 6)
(3989, 7) (998, 7)
(3989, 12) (998, 12)
(3989, 7) (998, 7)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 7) (998, 7)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 9) (998, 9)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 10) (998, 10)
(3989, 9) (998, 9)
(3989, 10) (998, 10)
(3989, 13) (998, 13)
(3989, 11) (998, 11)
(3989, 13) (998, 13)
(3989, 11) (998, 11)
(3989, 14) (998, 14)
(3989, 10) (998, 10)
(3989, 14) (998, 14)
(3989, 8) (998, 8)
(3989, 6) (998, 6)
(3989, 14) (998, 14)
(3989, 17) (998, 17)
(3989, 9) (998, 9)
(3989, 6) (998, 6)
(3989, 10) (998, 10)
(3989, 13) (998, 13)
(3989, 9) (998, 9)
(3989, 14) (998, 14)
(3989, 7) (998, 7)
(3989, 14) (998, 14)
(3989, 10) (998, 10)
(3989, 13) (998, 13)
(3989, 8) 

(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 9) (998, 9)
(3989, 9) (998, 9)
(3989, 10) (998, 10)
(3989, 10) (998, 10)
(3989, 9) (998, 9)
(3989, 12) (998, 12)
(3989, 14) (998, 14)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 13) (998, 13)
(3989, 9) (998, 9)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 7) (998, 7)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 6) (998, 6)
(3989, 11) (998, 11)
(3989, 14) (998, 14)
(3989, 15) (998, 15)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 9) (998, 9)
(3989, 12) (998, 12)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 17) (998, 17)
(3989, 5) (998, 5)
(3989, 10) (998, 10)
(3989, 5) (998, 5)
(3989, 13) (998, 13)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 8) (998, 8)
(3989, 10) (998, 10)
(3989, 5) (998, 5)
(3989, 7) (998, 7)
(3989, 9) (998, 9)
(3989, 9) (998, 9)
(3989, 12) (998, 12)
(3989, 9) (998, 9)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 9) (998, 9)
(3989, 11) (998, 11)
(39

(3989, 8) (998, 8)
(3989, 7) (998, 7)
(3989, 10) (998, 10)
(3989, 9) (998, 9)
(3989, 10) (998, 10)
(3989, 7) (998, 7)
(3989, 11) (998, 11)
(3989, 7) (998, 7)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 9) (998, 9)
(3989, 6) (998, 6)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 9) (998, 9)
(3989, 8) (998, 8)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 9) (998, 9)
(3989, 11) (998, 11)
(3989, 13) (998, 13)
(3989, 9) (998, 9)
(3989, 12) (998, 12)
(3989, 9) (998, 9)
(3989, 9) (998, 9)
(3989, 14) (998, 14)
(3989, 8) (998, 8)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 12) (998, 12)
(3989, 14) (998, 14)
(3989, 10) (998, 10)
(3989, 8) (998, 8)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 8) (998, 8)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 9) (998, 9)
(3989, 12) (998, 12)
(3989, 6) (998, 6)
(3989, 12) (998, 12)
(3989, 8) (998, 8)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 11

(3989, 8) (998, 8)
(3989, 8) (998, 8)
(3989, 9) (998, 9)
(3989, 8) (998, 8)
(3989, 13) (998, 13)
(3989, 13) (998, 13)
(3989, 13) (998, 13)
(3989, 12) (998, 12)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 13) (998, 13)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 14) (998, 14)
(3989, 5) (998, 5)
(3989, 16) (998, 16)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 13) (998, 13)
(3989, 12) (998, 12)
(3989, 14) (998, 14)
(3989, 12) (998, 12)
(3989, 7) (998, 7)
(3989, 13) (998, 13)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 14) (998, 14)
(3989, 8) (998, 8)
(3989, 12) (998, 12)
(3989, 8) (998, 8)
(3989, 12) (998, 12)
(3989, 12) (998, 12)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 10) (998, 10)
(3989, 9) (998, 9)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 15) (998, 15)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 9) (998, 9)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 12) (998, 12)
(3989, 8) (9

(3989, 13) (998, 13)
(3989, 8) (998, 8)
(3989, 9) (998, 9)
(3989, 10) (998, 10)
(3989, 10) (998, 10)
(3989, 11) (998, 11)
(3989, 11) (998, 11)
(3989, 12) (998, 12)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 11) (998, 11)
(3989, 10) (998, 10)
(3989, 10) (998, 10)
(3989, 8) (998, 8)
(3989, 13) (998, 13)
(3989, 9) (998, 9)
(3989, 12) (998, 12)
(3989, 11) (998, 11)
(3989, 13) (998, 13)
(3989, 10) (998, 10)
(3989, 13) (998, 13)
(3989, 12) (998, 12)
(3989, 10) (998, 10)
(3989, 9) (998, 9)
(3989, 11) (998, 11)
(3989, 8) (998, 8)
(3989, 12) (998, 12)
(3989, 14) (998, 14)
(3989, 10) (998, 10)
(3989, 10) (998, 10)
(3989, 14) (998, 14)
(3989, 12) (998, 12)
(3989, 10) (998, 10)
(3989, 12) (998, 12)
(3989, 14) (998, 14)
(3989, 14) (998, 14)
(3989, 9) (998, 9)
(3989, 8) (998, 8)
(3989, 12) (998, 12)
(3989, 11) (998, 11)
(3989, 13) (998, 13)
(3989, 13) (998, 13)
(3989, 13) (998, 13)
(3989, 13) (998, 13)
(3989, 7) (998, 7)
(3989, 10) (998, 10)
(3989, 15) (998, 15)
(3989, 11) (998, 11)
(3989, 11)

...

(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (9

(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
88 	118   
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(

(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
92 	114   
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(

(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 2) (998, 2)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (9

(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
99 	113   
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 3) (998, 3)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(3989, 4) (998, 4)
(